In [1]:
%reload_ext autoreload
%autoreload 2

import os, psutil

os.environ['NUMEXPR_MAX_THREADS'] = '20'
os.environ['NUMBA_DEBUGINFO'] = '0'

from alphadia.extraction import processlogger
processlogger.init_logging()
import logging

logger = logging.getLogger()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import neptune.new as neptune

#from alphabase.spectral_library.flat import SpecLibFlat
from alphabase.spectral_library.base import SpecLibBase
#from alphabase.spectral_library.reader import LibraryReaderBase

from alphadia.extraction.data import TimsTOFDIA
from alphadia.extraction.planning import Plan, Workflow
from alphadia.extraction.calibration import RunCalibration
from alphadia.extraction.candidateselection import MS1CentricCandidateSelection
from alphadia.extraction.scoring import fdr_correction, MS2ExtractionWorkflow
import alphadia.extraction.utils as utils
yaml_file = 'config.yaml'

raw_files = ['/Users/georgwallmann/Documents/data/alphadia_benchmarking/raw_data/diaPASEF_vs_synchroPASEF/20221221_TIMS05_PaSk_SA_HeLa_Evo05_200ng_21min_IM0713_diaPASEF_S4-A1_1_500.d',
             '/Users/georgwallmann/Documents/data/raw_data/Alpha_dia_benchmarking/diaPASEF_vs_synchroPASEF/20221221_TIMS05_PaSk_SA_HeLa_Evo05_200ng_21min_IM0713_diaPASEF_S4-A2_1_504.d',
             '/Users/georgwallmann/Documents/data/raw_data/Alpha_dia_benchmarking/diaPASEF_vs_synchroPASEF/20221221_TIMS05_PaSk_SA_HeLa_Evo05_200ng_21min_IM0713_diaPASEF_S4-A3_1_508.d',
             '/Users/georgwallmann/Documents/data/raw_data/Alpha_dia_benchmarking/diaPASEF_vs_synchroPASEF/20221221_TIMS05_PaSk_SA_HeLa_Evo05_200ng_21min_IM0713_diaPASEF_S4-A4_1_512.d',
             #'/Users/georgwallmann/Documents/data/raw_data/Alpha_dia_benchmarking/diaPASEF_vs_synchroPASEF/20221221_TIMS05_PaSk_SA_HeLa_Evo05_200ng_21min_IM0713_SyP_classical_5bins_S2-A1_1_449.d',
             #'/Users/georgwallmann/Documents/data/raw_data/Alpha_dia_benchmarking/diaPASEF_vs_synchroPASEF/20221221_TIMS05_PaSk_SA_HeLa_Evo05_200ng_21min_IM0713_SyP_classical_5bins_S2-A2_1_453.d',
             #'/Users/georgwallmann/Documents/data/raw_data/Alpha_dia_benchmarking/diaPASEF_vs_synchroPASEF/20221221_TIMS05_PaSk_SA_HeLa_Evo05_200ng_21min_IM0713_SyP_classical_5bins_S2-A3_1_457.d',
             #'/Users/georgwallmann/Documents/data/raw_data/Alpha_dia_benchmarking/diaPASEF_vs_synchroPASEF/20221221_TIMS05_PaSk_SA_HeLa_Evo05_200ng_21min_IM0713_SyP_classical_5bins_S2-A4_1_464.d'

             ]

output_location = '/Users/georgwallmann/Documents/data/alphadia_benchmarking/alphadia_runs'

try:
    neptune_token = os.environ['NEPTUNE_TOKEN']
except KeyError:
    logger.error('NEPTUNE_TOKEN environtment variable not set')


0:00:02.432814  WARNING: WARNING: Temp mmap arrays are written to /var/folders/lc/9594t94d5b5_gn0y04w1jh980000gn/T/temp_mmap_0n9d80vt. Cleanup of this folder is OS dependant, and might need to be triggered manually! Current space: 1,192,358,027,264
0:00:02.433342  WARNING: WARNING: No Bruker libraries are available for this operating system. Mobility and m/z values need to be estimated. While this estimation often returns acceptable results with errors < 0.02 Th, huge errors (e.g. offsets of 6 Th) have already been observed for some samples!
0:00:02.433589  INFO: 


In [2]:
test_lib = SpecLibBase()
test_lib_location = '/Users/georgwallmann/Documents/data/alphadia_benchmarking/libraries/marvin_scp/MSfragger_library_mod_noLossType_d0_d4_d8_shared_eg.hdf'
test_lib.load_hdf(test_lib_location, load_mod_seq=True)


In [3]:
plan = Plan(raw_files)
plan.from_spec_lib_base(test_lib)
#plan.run(output_location, keep_decoys=True, fdr=1.0)
for dia_data, precursors_flat, fragments_flat in plan.get_run_data():
    break

0:00:05.772048  PROGRESS:       _   _      _         ___ ___   _   
0:00:05.772446  PROGRESS:      /_\ | |_ __| |_  __ _|   \_ _| /_\  
0:00:05.772693  PROGRESS:     / _ \| | '_ \ ' \/ _` | |) | | / _ \ 
0:00:05.772924  PROGRESS:    /_/ \_\_| .__/_||_\__,_|___/___/_/ \_\
0:00:05.773112  PROGRESS:            |_|                            
0:00:05.773310  PROGRESS: 
0:00:05.773878  INFO: loading default config from /Users/georgwallmann/Documents/git/alphadia/alphadia/extraction/../../misc/config/default.yaml
0:00:05.778967  PROGRESS: version: 1.0.1
0:00:05.779792  PROGRESS: hostname: PCMANN428.biochem.mpg.de
0:00:05.780425  PROGRESS: date: 2023-03-09 17:46:48
0:00:09.901142  INFO: renaming precursor_columns columns
0:00:09.901978  INFO: renaming fragment_columns columns
0:00:09.902347  INFO: ========= Library Stats =========
0:00:09.902534  INFO: Number of precursors: 275,196
0:00:09.931512  INFO: 	thereof targets:137,601
0:00:09.931916  INFO: 	thereof decoys: 137,595
0:00:09.933568  IN

In [50]:
from alphadia.extraction import hybridselection

selection = hybridselection.HybridCandidateSelection(
    dia_data,
    precursors_flat[:1000],
    fragments_flat,
    rt_tolerance=360,
    mz_tolerance=30,
    mobility_tolerance=0.05,
    candidate_count=3,
    thread_count=20,
    #debug=True,
)

egs = selection()

0:13:11.517911  INFO: Duty cycle consists of 13 frames, 1.39 seconds cycle time
0:13:11.518295  INFO: Duty cycle consists of 928 scans, 0.00065 1/K_0 resolution
0:13:11.518575  INFO: Expected peak length in RT is 10.00 seconds, sigma is 1.80
0:13:11.519029  INFO: Expected peak length in mobility is 0.03 1/K_0, sigma is 11.60
100%|██████████| 169/169 [00:05<00:00, 32.60it/s] 


In [51]:
from alphadia.extraction.numba import fragments

In [52]:
prior_abundance = np.array([1, 1, 1, 1, 1, 1])

score_grouped = False
exclude_shared_fragments = True

if score_grouped:
    score_groups = egs[0].decoy

else:
    score_groups = np.arange(len(egs[0].decoy))

fragments.get_ion_group_mapping(
    egs[0].fragments.precursor_idx, 
    egs[0].fragments.mz,
    egs[0].fragments.intensity,
    prior_abundance,
    score_groups,
    exclude_shared=exclude_shared_fragments
)

(array([213.15976, 217.18486, 341.21832, 345.24344, 349.2627 , 646.3559 ,
        650.381  , 654.40027], dtype=float32),
 array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         1.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 1.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 1.        ],
        [0.59397462, 0.        , 1.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.63156048, 0.        , 1.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 1.        ,
         0.        , 0.        , 0.        ]]))

In [53]:
egs[0].isotope_mz

isotope_mz = egs[0].isotope_mz.flatten()
isotope_intensity = egs[0].isotope_intensity.flatten()
isotope_precursor = np.repeat(np.arange(0, egs[0].isotope_mz.shape[0]), egs[0].isotope_mz.shape[1])

order = np.argsort(isotope_mz)
isotope_mz = isotope_mz[order]
isotope_intensity = isotope_intensity[order]
isotope_precursor = isotope_precursor[order]

In [54]:
isotope_mz

array([410.73743, 411.2391 , 411.74078, 412.74997, 413.25165, 413.75333,
       414.7596 , 414.7687 , 415.2613 , 415.2704 , 415.76297, 415.77206,
       416.78128, 417.28296, 417.78464, 418.7909 , 419.29257, 419.79425],
      dtype=float32)

In [55]:
isotope_intensity

array([0.62718904, 0.28260028, 0.07386316, 0.62718904, 0.28260028,
       0.07386316, 0.62718904, 0.62718904, 0.28260028, 0.28260028,
       0.07386316, 0.07386316, 0.62718904, 0.28260028, 0.07386316,
       0.62718904, 0.28260028, 0.07386316], dtype=float32)

In [56]:
isotope_precursor

array([0, 0, 0, 1, 1, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4, 5, 5, 5])

In [ ]:
fragments.get_ion_group_mapping(
    isotope_precursor, 
    isotope_mz,
    isotope_intensity,
    prior_abundance,
    score_groups,
    exclude_shared=exclude_shared_fragments
)

In [37]:
import numba as nb

os.environ['DEBUG'] = 'True'

from alphadia.extraction.hybridselection import HybridElutionGroup, HybridElutionGroupContainer

def assemble_elution_groups(
        precursors_flat,
        rt_column='rt_library',
        mobility_column='mobility_library',
        precursor_mz_column='mz_library'
        ):
    
    """
    Assemble elution groups from precursor library.

    Parameters
    ----------

    precursors_flat : pandas.DataFrame
        Precursor library.

    rt_column : str
        Column name of the retention time values.

    mobility_column : str
        Column name of the mobility values.

    precursor_mz_column : str
        Column name of the precursor m/z values.

    Returns
    -------
    HybridElutionGroupContainer
        Numba jitclass with list of elution groups.
    """
    
    if len(precursors_flat) == 0:
        return

    available_isotopes = utils.get_isotope_columns(precursors_flat.columns)
    available_isotope_columns = [f'i_{i}' for i in available_isotopes]

    precursors_sorted = precursors_flat.sort_values('elution_group_idx').copy()

    @nb.njit(debug=True)
    def assemble_njit(
        elution_group_idx,
        precursor_idx,
        channel,
        flat_frag_start_stop_idx,
        rt_values,
        mobility_values,
        charge,
        decoy,
        precursor_mz,
        isotope_intensity
    ):
        elution_group = elution_group_idx[0]
        elution_group_start = 0
        elution_group_stop = 0

        eg_list = []
        
        while elution_group_stop < len(elution_group_idx)-1:
            
            elution_group_stop += 1

            if elution_group_idx[elution_group_stop] != elution_group:
                    
                eg_list.append(HybridElutionGroup(    
                    elution_group,
                    precursor_idx[elution_group_start:elution_group_stop],
                    channel[elution_group_start:elution_group_stop],
                    flat_frag_start_stop_idx[elution_group_start:elution_group_stop],
                    rt_values[elution_group_start],
                    mobility_values[elution_group_start],
                    charge[elution_group_start],
                    decoy[elution_group_start:elution_group_stop],
                    precursor_mz[elution_group_start:elution_group_stop],
                    isotope_intensity[elution_group_start:elution_group_stop]
                ))

                elution_group_start = elution_group_stop
                elution_group = elution_group_idx[elution_group_start]
                
        egs = nb.typed.List(eg_list)
        return HybridElutionGroupContainer(egs)

    return assemble_njit(
        precursors_sorted['elution_group_idx'].values.astype(np.uint32),
        precursors_sorted['precursor_idx'].values.astype(np.uint32),
        precursors_sorted['channel'].values.astype(np.uint32),
        precursors_sorted[['flat_frag_start_idx','flat_frag_stop_idx']].values.copy().astype(np.uint32),
        precursors_sorted[rt_column].values.astype(np.float64),
        precursors_sorted[mobility_column].values.astype(np.float64),
        precursors_sorted['charge'].values.astype(np.uint8),
        precursors_sorted['decoy'].values.astype(np.uint8),
        precursors_sorted[precursor_mz_column].values.astype(np.float64),
        precursors_sorted[available_isotope_columns].values.copy().astype(np.float32),
    )
    

In [38]:
assemble_elution_groups(precursors_flat)

/Users/georgwallmann/miniconda3/envs/alpha/lib/python3.9/site-packages/numba/core/lowering.py:107: NumbaDebugInfoWarning: Could not find source for function: <function assemble_elution_groups.<locals>.assemble_njit at 0x2eb735dc0>. Debug line information may be inaccurate.
  warnings.warn(NumbaDebugInfoWarning(msg))


In [ ]:
%timeit assemble_elution_groups(precursors_flat)

In [ ]:
from alphadia.extraction import candidateselection

gaussian = candidateselection.GaussianFilter(dia_data)
kernel = gaussian.get_kernel()

In [ ]:
plt.imshow(kernel)

In [ ]:
kernel[:,10:].shape

In [ ]:
p = dense_precursors[0,0,:,:32,:64]
p.shape

In [ ]:
x = np.tile(p, (10,1,1,1))

In [ ]:
x.shape

In [ ]:
%timeit smooth = fourier_a1(x, kernel)

In [ ]:
plt.imshow(p[0])

In [ ]:
plt.imshow(smooth[0])

In [ ]:
plt.imshow(roll(smooth[0,0],-10,-10))

In [ ]:
import numba as nb
import numpy as np

@nb.njit
def jit_fft(x):
    return np.fft.fft(x)

a = np.array([2, 7, 1, 8, 2, 8, 1, 8])
jit_fft(a)

In [ ]:
from alphadia.extraction import quadrupole, scoring, features
q = quadrupole.SimpleQuadrupole(dia_data.cycle)

In [ ]:
from alphadia.extraction import scoring

extraction = scoring.MS2ExtractionWorkflow(
    dia_data,
    precursors_flat,
    fragments_flat,
    candidates,
    q,
    precursor_mz_tolerance=30,
    fragment_mz_tolerance=30
)

feature_df, fragment_df = extraction()

In [ ]:
df = scoring.fdr_correction(feature_df)

In [ ]:
df = scoring.fdr_correction(df)